# Runpod Dev Environment Setup

SSH implements public-key cryptography to establish trust between client and server systems. In this paradigm, we have the **public key** (`.pub`) that functions as an access *verifier*, installed on target systems (like Runpod nodes), and the **private key** serves as the unique authentication factor for mathematically proving identity.

Below, we create public keys which we install onto Runpod and GitHub systems. The corresponding private key for Runpod is stored locally, while that for GitHub is securely copied via `scp`. This gives every new pod read and write access to GitHub repositories.

![](./img/runpod-ssh.png)

## Give local SSH access to pods

1. Generate SSH keys for RunPod (no passphrase):
```bash
ssh-keygen -t ed25519 -C "runpod" -f ~/.ssh/runpod -N ""
cat ~/.ssh/runpod.pub
```
2. Add this in `Settings>SSH Public Keys` (separated by newline).

## Give pods SSH access to GitHub
1. Generate SSH keys to access GitHub from each pod (no passphrase):
```bash
ssh-keygen -t ed25519 -C "runpod-github" -f ~/.ssh/runpod_github -N ""
cat ~/.ssh/runpod_github.pub
```
2. Register the printed public SSH key in Github.
3. Copy the pod **SSH over exposed TCP** connection string[^connstring] and paste below:
```bash
export RUNPOD_SSH=""
```
4. Verify the captured IP and port number:
```bash
# Extract environmental variables
export RUNPOD_IP=$(echo "$RUNPOD_SSH" | sed -E 's/.*@([0-9.]+).*/\1/')
export RUNPOD_PORT=$(echo "$RUNPOD_SSH" | sed -E 's/.*-p ([0-9]+).*/\1/')

echo $RUNPOD_IP
echo $RUNPOD_PORT
```
5. Copy the GitHub SSH key to the pod and prepare the pod's SSH environment:
```bash
scp -P $RUNPOD_PORT -i ~/.ssh/runpod ~/.ssh/runpod_github root@$RUNPOD_IP:/root/.ssh/runpod_github

ssh -p $RUNPOD_PORT -i ~/.ssh/runpod root@$RUNPOD_IP '
    # Set secure permissions for SSH directory and key
    chmod 700 ~/.ssh                                                            # <1>
    chmod 600 ~/.ssh/runpod_github                                              # <2>

    # Add GitHub to known_hosts and set permissions
    ssh-keyscan github.com >> ~/.ssh/known_hosts                                # <3>
    chmod 644 ~/.ssh/known_hosts                                                # <4>

    # Start SSH agent and load the key
    ssh-agent -s > ~/.ssh-agent-info                                            # <5>
    source ~/.ssh-agent-info
    ssh-add ~/.ssh/runpod_github                                                # <6>

    # Ensure the agent is auto-loaded in future sessions
    line="if [ -f ~/.ssh-agent-info ]; then source ~/.ssh-agent-info; fi"
    grep -qxF "$line" ~/.bashrc || echo "$line" >> ~/.bashrc                    # <7>
'
```
1. Permissions: only owner can read, write, or execute directory.
2. Permissions: only owner can read or write on the private key file.
3. Get GitHub's SSH host key and appends to `known_hosts` to avoid authenticity prompts.
4. Permissions: `known_hosts` owner can read/write, others have read-only access.
5. Run SSH agent in the background and output env vars to `~/.ssh-agent-info` for later sourcing.
6. Add GitHub private key to SSH agent. 
7. Checks if the source command exists as a whole line in `~/.bashrc` (`grep -qxF` quiet, entire line, no regex), else appends it to the end of the file, ensuring the line is only added once. This ensures that environmental variables are loaded each time you SSH into the pod.

:::{.callout-note}
The `ssh-agent` process persists in the background once started, holding your loaded keys in memory. What does not persist are the environment variables (`SSH_AUTH_SOCK`, `SSH_AGENT_PID`) that let your shell know how to communicate with that agent. When you open a new shell or SSH session, those env vars are not automatically set, so your shell can’t find the running agent unless you restore them (e.g. by sourcing `~/.ssh-agent-info`).
:::

6. SSH into pod and add SSH keys:
```bash
ssh root@$RUNPOD_IP -p $RUNPOD_PORT -i ~/.ssh/runpod
```

[^connstring]: e.g. `export RUNPOD_SSH="ssh root@63.141.33.33 -p 22011 -i ~/.ssh/id_ed25519"`